In [42]:
import pandas as pd

df = pd.read_csv("example/a1_raport.csv")

In [43]:
df.head()

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,sredn._martinamm,sredn._sitowamm,srednia_jaskrawosc,r,g,b,y,liczenie_obiektow_w,udzial_punktow,pole_obrazu_mm2
0,1,292.0,699.0,29.2,69.9,0.92,1.3,0.8,69.24,4.087006,...,1.3,1.079358,128.0,97.5,139.8,147.2,128.0,1,12.2,9.0
1,2,572.0,615.5,57.2,61.6,1.31,1.4,1.3,115.36,4.262742,...,1.4,1.291489,139.7,105.9,152.5,162.4,139.7,1,13.4,11.2
2,3,971.0,137.5,97.1,13.8,0.65,1.1,0.7,61.01,3.155635,...,1.1,0.906223,120.6,88.9,132.7,141.1,120.6,1,9.8,8.1
3,4,867.5,241.5,86.8,24.2,0.28,0.7,0.6,81.50,2.072792,...,0.7,0.591727,143.1,112.4,155.3,160.9,143.1,1,5.5,6.8
4,5,833.5,535.5,83.4,53.6,0.97,1.4,1.1,21.72,4.062742,...,1.4,1.111325,130.6,98.4,143.3,149.4,130.6,1,12.0,9.6


In [44]:
# Scaling
df['dlugoscmm'] = df['dlugoscmm']/1.49
df['szerokoscmm'] = df['szerokoscmm']/1.49
df['powierzchniamm'] = df['powierzchniamm']/2.235

In [45]:
# Set proble volume in mL
PROBE_VOLUME_ML = 6

In [46]:
df.shape

(50, 23)

In [47]:
df = df[df['dlugoscmm'] >= 0.2]
df = df[df['szerokoscmm'] <= 1.5]

In [48]:
df.shape

(49, 23)

In [49]:
df['Pw'] = 3.14*(df['szerokoscmm']/2)**2+df['szerokoscmm']*(df['dlugoscmm']-df['szerokoscmm'])

In [50]:
df['R'] = df['Pw'] / df['powierzchniamm']

In [51]:
df['Dk'] = df['dlugoscmm']
df['Sk'] = df['szerokoscmm']
df['D/S'] = df['Dk'] / df['Sk']

In [52]:
bacteria_types = []
Dks = []
Sks = []
for idx,row  in df.iterrows():
    if row['R'] > 1.2:
        Dks.append(row['szerokoscmm'])
        Sks.append(row['Pw']*0.8)
        bacteria_types.append("Krzywe")
    else:
        Dks.append(row['dlugoscmm'])
        Sks.append(row['szerokoscmm'])
        if row['D/S'] > 1.5:
            bacteria_types.append("Pałeczki")
        else:
            bacteria_types.append("Ziarniaki")
df['bacteria_type'] = bacteria_types
df['Dk'] = Dks
df['Sk'] = Sks

In [53]:
df['Ob'] =(3.14*(df['Sk']**3)/6)+(3.14*((df['Sk']**2)/4)*(df['Dk']-df['Sk']))

In [54]:
df['biomasa'] =((104.5*(df['Ob']**0.59))*0.86)/1000000

In [55]:
df['bialko'] = 104.5 * (df['Ob']**0.59)

In [56]:
df['wegiel'] = 0.86

In [57]:
df['bacteria_type'].value_counts()

bacteria_type
Pałeczki     18
Ziarniaki    16
Krzywe       15
Name: count, dtype: int64

In [58]:
df.groupby("bacteria_type")

In [59]:
bins = [0, 0.1, 0.2, 0.5, 1.0, float("inf")]
labels = ["<=0.1", "0.1–0.2", "0.2–0.5", "0.5–1.0", ">1.0"]

df["Ob_bucket"] = pd.cut(df["Ob"], bins=bins, labels=labels, right=True)

grouped = df.groupby(["bacteria_type", "Ob_bucket"])
result = grouped.size().reset_index(name="count")

/tmp/ipykernel_10361/3292400346.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(["bacteria_type", "Ob_bucket"])


In [60]:
result

,bacteria_type,Ob_bucket,count
0,Krzywe,<=0.1,8
1,Krzywe,0.1–0.2,3
2,Krzywe,0.2–0.5,2
3,Krzywe,0.5–1.0,1
4,Krzywe,>1.0,0
5,Pałeczki,<=0.1,6
6,Pałeczki,0.1–0.2,4
7,Pałeczki,0.2–0.5,5
8,Pałeczki,0.5–1.0,1
9,Pałeczki,>1.0,2


In [61]:
import numpy as np
total_bacteria_count = np.sum(result['count'])

In [62]:
scale_to_1_L = 1000 / PROBE_VOLUME_ML
result['count_in_1_L'] = result['count'] * scale_to_1_L

In [63]:

result['bio_diversity'] =((result['count']+1)/total_bacteria_count)*np.log((result['count']+1)/total_bacteria_count)

In [64]:

result['bio_diversity_1_L'] =((result['count_in_1_L']+1)/(total_bacteria_count*scale_to_1_L))*np.log((result['count_in_1_L']+1)/(total_bacteria_count*scale_to_1_L))

In [65]:
result

,bacteria_type,Ob_bucket,count,count_in_1_L,bio_diversity,bio_diversity_1_L
0,Krzywe,<=0.1,8,1333.333333,-0.313871,-0.298726
1,Krzywe,0.1–0.2,3,500.000000,-0.207076,-0.173508
2,Krzywe,0.2–0.5,2,333.333333,-0.173287,-0.132691
3,Krzywe,0.5–1.0,1,166.666667,-0.132419,-0.081009
4,Krzywe,>1.0,0,0.000000,-0.080650,-0.001123
5,Pałeczki,<=0.1,6,1000.000000,-0.280772,-0.260065
6,Pałeczki,0.1–0.2,4,666.666667,-0.235600,-0.207261
7,Pałeczki,0.2–0.5,5,833.333333,-0.259930,-0.235758
8,Pałeczki,0.5–1.0,1,166.666667,-0.132419,-0.081009
9,Pałeczki,>1.0,2,333.333333,-0.173287,-0.132691


In [66]:
shannon_index = np.sum(result['bio_diversity'])*-1
shannon_index_1_L = np.sum(result['bio_diversity_1_L'])*-1
print(f"Shannon index: {shannon_index}")
print(f"Shannon index 1 L: {shannon_index_1_L}")

Shannon index: 2.972205819846199
Shannon index 1 L: 2.3813815848769715
